# Predict Composite Cognitive Score 
Predict the composite cognitive score of a set of ADNI patients using Random Forrest and SVM methods. 

We are using the four ADSP-PHC composite scores for *Memory, Executive Function, Language and Visuospatial Ability*. The methods for deriving these are described in 'ADSP Phenotype Harmonization Consortium – Derivation of Cognitive Composite Scores' by Mukherjee et al (https://ida.loni.usc.edu/download/files/study/083f5b49-98d1-494a-aaf1-3310a9a8e62c/file/adni/ADNI_Cognition_Methods_Psychometric_Analyses_Oct2022.pdf).

In [56]:
import pandas as pd
import os
import numpy as np
from scipy.io import loadmat

# scikit-learn modules
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor 

import matplotlib.pyplot as plt 

## Processing Data

Match up the composite cognitive scores and functional connectivity data, then split into test + training sets

In [57]:
ADSP_DATA_PATH = "../data/ADSP_PHC_COGN_Dec2023_FILTERED.csv"
FC_DATA_PATH = "../../FMRI_ADNI_DATA/fc/"
ADAS_DATA_PATH = "../data/ADAS_ADNIGO23_17Apr2024.csv"

In [58]:
# Process the ADSP Data

# adsp_df = pd.read_csv(ADSP_DATA_PATH)
adsp_df = pd.read_csv(ADAS_DATA_PATH)
adsp_df = adsp_df.drop(columns=adsp_df.columns[0])
adsp_df.head()

,ID,PTID,RID,SITEID,VISCODE,VISCODE2,VISDATE,USERDATE,USERDATE2,WORDLIST,...,TOTSCORE,Q13UNABLE,Q13TASKA,Q13TASKB,Q13TASKC,Q13SCORE,TOTAL13,DATE,SOURCE,update_stamp
0,4,023_S_0376,376,17,m48,m48,2010-05-14,2010-06-02,2011-02-28,1.0,...,9.0,NaN,28.0,0.0,0.0,0.0,16.0,NaN,NaN,2012-11-16 09:32:21.0
1,14,016_S_1117,1117,12,m36,m36,2010-06-24,2010-06-27,2011-02-28,1.0,...,24.0,NaN,5.0,0.0,0.0,4.0,38.0,NaN,NaN,2012-11-16 09:32:21.0
2,16,016_S_1326,1326,12,m36,m36,2010-05-12,2010-06-28,2011-11-08,1.0,...,23.0,NaN,16.0,0.0,1.0,2.0,35.0,NaN,NaN,2012-11-16 09:32:21.0
3,18,031_S_0618,618,21,m48,m48,2010-06-25,2010-06-28,2011-02-28,1.0,...,3.0,NaN,24.0,0.0,0.0,0.0,4.0,NaN,NaN,2012-11-16 09:32:21.0
4,20,035_S_0292,292,24,m48,m48,2010-04-21,2010-06-30,2011-02-28,1.0,...,19.0,NaN,21.0,0.0,0.0,1.0,30.0,NaN,NaN,2012-11-16 09:32:21.0


In [59]:
adas = adsp_df.dropna(subset=['VISCODE2', 'TOTSCORE'])
adas.head()

,ID,PTID,RID,SITEID,VISCODE,VISCODE2,VISDATE,USERDATE,USERDATE2,WORDLIST,...,TOTSCORE,Q13UNABLE,Q13TASKA,Q13TASKB,Q13TASKC,Q13SCORE,TOTAL13,DATE,SOURCE,update_stamp
0,4,023_S_0376,376,17,m48,m48,2010-05-14,2010-06-02,2011-02-28,1.0,...,9.0,NaN,28.0,0.0,0.0,0.0,16.0,NaN,NaN,2012-11-16 09:32:21.0
1,14,016_S_1117,1117,12,m36,m36,2010-06-24,2010-06-27,2011-02-28,1.0,...,24.0,NaN,5.0,0.0,0.0,4.0,38.0,NaN,NaN,2012-11-16 09:32:21.0
2,16,016_S_1326,1326,12,m36,m36,2010-05-12,2010-06-28,2011-11-08,1.0,...,23.0,NaN,16.0,0.0,1.0,2.0,35.0,NaN,NaN,2012-11-16 09:32:21.0
3,18,031_S_0618,618,21,m48,m48,2010-06-25,2010-06-28,2011-02-28,1.0,...,3.0,NaN,24.0,0.0,0.0,0.0,4.0,NaN,NaN,2012-11-16 09:32:21.0
4,20,035_S_0292,292,24,m48,m48,2010-04-21,2010-06-30,2011-02-28,1.0,...,19.0,NaN,21.0,0.0,0.0,1.0,30.0,NaN,NaN,2012-11-16 09:32:21.0


In [60]:
# adsp_df = adsp_df.drop(columns=[
#     'SUBJID', 'PHASE', 'VISCODE', 'EXAMDATE', 'PHC_Visit', 'PHC_Sex', 'PHC_Education', 'PHC_Ethnicity', 'PHC_Race', 'PHC_Age_Cognition', 
#     'PHC_MEM_SE', 'PHC_MEM_PreciseFilter', 'PHC_EXF_SE', 'PHC_EXF_PreciseFilter', 'PHC_LAN_SE', 'PHC_LAN_PreciseFilter', 'PHC_VSP_SE',
#     'PHC_VSP_PreciseFilter'
# ])
# adsp_df.head()
adas = adas[['RID', 'VISCODE2', 'TOTSCORE']]
adas.head()

,RID,VISCODE2,TOTSCORE
0,376,m48,9.0
1,1117,m36,24.0
2,1326,m36,23.0
3,618,m48,3.0
4,292,m48,19.0


In [61]:
adas['VISCODE2'].unique()

array(['m48', 'm36', 'bl', 'm60', 'm06', 'm12', 'm72', 'm24', 'm84',
       'm96', 'm108', 'm120', 'm132', 'm144', 'm102', 'm30', 'm114',
       'm126', 'm186', 'm54', 'm138', 'm156', 'm180', 'm42', 'm18',
       'm168', 'm174', 'm150', 'm162', 'm78', 'm90', 'm66', 'm192',
       'm198', 'm204'], dtype=object)

In [81]:
# def replace_viscode(str):
#     if str == 'BL' or str == 'SC':
#         return adsp_df['VISCODE2'].replace(str, 'M000')
#     else:
#         vis = str[1:]
#         vis = vis.zfill(3)
#         vis = 'M' + vis
#         return adsp_df['VISCODE2'].replace(str, vis)

# adsp_df['VISCODE2'] = adsp_df['VISCODE2'].str.upper()

# # Pad the visit codes
# for val in adsp_df['VISCODE2'].unique():
#     print(val)
# #     adsp_df['VISCODE2'] = replace_viscode(val)

# # Pad the RID values
# # adsp_df['RID'] = adsp_df['RID'].apply(lambda x: str(x).zfill(4))

# adsp_df.head()
def replace_viscode(str):
    if str == 'BL' or str == 'SC':
        return adas['VISCODE2'].replace(str, 'M000')
    else:
        vis = str[1:]
        vis = vis.zfill(3)
        vis = 'M' + vis
        return adas['VISCODE2'].replace(str, vis)

adas['VISCODE2'] = adas['VISCODE2'].str.upper()

# Pad the visit codes
for val in adas['VISCODE2'].unique():
#     print(val)
    adas['VISCODE2'] = replace_viscode(val)

# Pad the RID values
adas['RID'] = adas['RID'].apply(lambda x: str(x).zfill(4))

# adas.head()
adas.head()

,RID,VISCODE2,TOTSCORE,FC_DATA
0,0376,M048,9.0,None
1,1117,M036,24.0,None
2,1326,M036,23.0,None
3,0618,M048,3.0,None
4,0292,M048,19.0,None


Get the FC data and add

In [50]:
import re

def get_rid_viscode(filename):
    pattern = r'sub-ADNI\d+S(\d{4})_ses-(M\d{3})'
    match = re.search(pattern, filename)

    if match:
        rid = match.group(1)
        viscode = match.group(2)
        return rid, viscode        
    else:
        print("Pattern not found in the filename.")
        return None


In [70]:
adas['FC_DATA'] = None

fc_dir = os.listdir(FC_DATA_PATH)

fc_files = [os.path.join(FC_DATA_PATH, file) for file in fc_dir if file.endswith('.mat')]
len(fc_files)

1478

In [76]:
adas.head()

,RID,VISCODE2,TOTSCORE,FC_DATA
0,0376,M048,9.0,None
1,1117,M036,24.0,None
2,1326,M036,23.0,None
3,0618,M048,3.0,None
4,0292,M048,19.0,None


In [90]:
for fc in fc_files:
    rid, viscode = get_rid_viscode(fc)
    adas.loc[(adas['RID'] == rid) & (adas['VISCODE2'] == viscode), 'FC_DATA'] = fc

In [92]:
adas[(adas['RID'] == str(4268))]

,RID,VISCODE2,TOTSCORE,FC_DATA
892,4268,M000,9.00,../../FMRI_ADNI_DATA/fc/sub-ADNI013S4268_ses-M...
1454,4268,M006,2.00,../../FMRI_ADNI_DATA/fc/sub-ADNI013S4268_ses-M...
2058,4268,M012,7.00,../../FMRI_ADNI_DATA/fc/sub-ADNI013S4268_ses-M...
3311,4268,M024,6.00,None
4092,4268,M036,6.00,None
6478,4268,M084,10.67,../../FMRI_ADNI_DATA/fc/sub-ADNI013S4268_ses-M...


In [95]:
adas_filtered = adas.dropna(subset=['FC_DATA'])
adas_filtered.shape

(1352, 4)

In [96]:
adas_filtered = adas_filtered.drop(adas_filtered[adas_filtered['VISCODE2'] == 'M162'].index)
adas_filtered = adas_filtered.drop(adas_filtered[adas_filtered['VISCODE2'] == 'M174'].index)
adas_filtered = adas_filtered.drop(adas_filtered[adas_filtered['VISCODE2'] == 'M180'].index)
adas_filtered = adas_filtered.drop(adas_filtered[adas_filtered['VISCODE2'] == 'M186'].index)
adas_filtered = adas_filtered.drop(adas_filtered[adas_filtered['VISCODE2'] == 'M192'].index)
adas_filtered.shape

(1342, 4)

In [98]:
# Save the adsp_df_filtered dataframe as a file
# adas_filtered.to_csv('../data/ADAS_ADNIGO23_17Apr2024_FILTERED_wfiles.csv')

In [13]:
# Get the FC data as numpy arrays
dim_x = len(adsp_df_filtered['FC_DATA'])
features = np.zeros(shape=(dim_x, 100, 200)) # get the first 100 regions

for i, file in enumerate(adsp_df_filtered['FC_DATA'].values):
    arr = loadmat(file)['ROI_activity'][:100, :] # get the first 100 regions
    if arr.shape[1] != 200:
        # add padding to get a constant shape
        diff = 200 - arr.shape[1]
        if diff < 0:
            arr = arr[:, :200]
            padded_array = arr
        else:
            pad_width = ((0, 0), (0, diff))  
            padded_array = np.pad(arr, pad_width, mode='constant', constant_values=0)
    features[i] = padded_array
features.shape

(1343, 100, 200)

In [14]:
y = adsp_df_filtered[['PHC_MEM', 'PHC_EXF', 'PHC_LAN', 'PHC_VSP']]
y.head()

,PHC_MEM,PHC_EXF,PHC_LAN,PHC_VSP
10,1.377,-0.092,0.666,0.963
91,0.902,0.579,0.757,0.264
119,0.645,0.525,0.448,-0.041
132,1.134,0.149,1.011,0.264
133,1.138,0.501,0.710,0.963


In [15]:
features

array([[[-25049.14050633, -25085.50632911, -25060.02531646, ...,
              0.        ,      0.        ,      0.        ],
        [-14151.73890339, -13546.8259356 , -13982.38294169, ...,
              0.        ,      0.        ,      0.        ],
        [-18845.32946146, -18463.66631468, -18761.41077086, ...,
              0.        ,      0.        ,      0.        ],
        ...,
        [-17233.40803213, -17272.77429719, -17155.16064257, ...,
              0.        ,      0.        ,      0.        ],
        [-20719.6733871 , -20669.24193548, -20702.26612903, ...,
              0.        ,      0.        ,      0.        ],
        [-20043.74358974, -20011.4017094 , -19964.002442  , ...,
              0.        ,      0.        ,      0.        ]],

       [[-25049.14050633, -25085.50632911, -25060.02531646, ...,
              0.        ,      0.        ,      0.        ],
        [-14151.73890339, -13546.8259356 , -13982.38294169, ...,
              0.        ,      0.     

In [ ]:
# split into test + training (80% train, 20% test)
features_2d = features.reshape(features.shape[0], -1)
x_train, x_test, y_train, y_test = train_test_split(features_2d, y, test_size = 0.2, random_state = 28)

## Random Forrest Method
Prediction not differentiable wrt to input - need a model per composite (memory, executive function, language and visuospatial)

In [16]:
split targets into the different composites

y_train_mem, y_train_exf, y_train_lan, y_train_vsp = y_train['PHC_MEM'], y_train['PHC_EXF'], y_train['PHC_LAN'], y_train['PHC_VSP']
y_test_mem, y_test_exf, y_test_lan, y_test_vsp = y_test['PHC_MEM'], y_test['PHC_EXF'], y_test['PHC_LAN'], y_test['PHC_VSP']

#### Memory Model

In [17]:
# MEMORY MODEL

# Remove NaNs in target
y_train_mem = y_train['PHC_MEM'].reset_index(drop=True)
y_test_mem =y_test['PHC_MEM'].reset_index(drop=True)

nan_indices = y_train_mem.index[y_train_mem.isna()]
y_train_mem = y_train_mem.drop(nan_indices)
x_train_mem = np.delete(x_train, nan_indices, axis = 0)
# print(nan_indices)

nan_indices_test = y_test_mem.index[y_test_mem.isna()]
y_test_mem = y_test_mem.drop(nan_indices_test)
x_test_mem = np.delete(x_test, nan_indices_test, axis = 0)

In [18]:
# Initializing the Random Forest Regression model with 10 decision trees
base_model_mem = RandomForestRegressor(n_estimators = 10, random_state = 5)

# Fitting the Random Forest Regression model to the data
base_model_mem.fit(x_train_mem, y_train_mem)

In [19]:
# x_test_mem.shape
# Predicting the target values of the test set
base_y_pred_mem = base_model_mem.predict(x_test_mem)

In [20]:
# from sklearn.metrics import r2_score

base_r2_mem = r2_score(y_test_mem, base_y_pred_mem)
print("Baseline R2 (MEM): ", base_r2_mem)

In [21]:
print(base_model_mem.get_params())

#### Random Search for best Hyperparameters

In [22]:
PREDICTOR_TYPE = 'EXF'

y_train_cleaned = y_train[f'PHC_{PREDICTOR_TYPE}'].reset_index(drop=True)
y_test_cleaned =y_test[f'PHC_{PREDICTOR_TYPE}'].reset_index(drop=True)

nan_indices = y_train_cleaned.index[y_train_cleaned.isna()]
y_train_cleaned = y_train_cleaned.drop(nan_indices)
x_train_cleaned = np.delete(x_train, nan_indices, axis = 0)

nan_indices_test = y_test_cleaned.index[y_test_cleaned.isna()]
y_test_cleaned = y_test_cleaned.drop(nan_indices_test)
x_test_cleaned = np.delete(x_test, nan_indices_test, axis = 0)

In [23]:
import json
from joblib import dump

# Get the best parameter set
best_params = rf_random.best_params_
PARAM_FILE = f'{PREDICTOR_TYPE}_best_params.json'

# Write data to a JSON file
with open(PARAM_FILE, 'w') as json_file:
    json.dump(best_params, json_file)
    
print("\n The best estimator across ALL searched params:\n", rf_random.best_estimator_)
print("\n The best score across ALL searched params:\n", rf_random.best_score_)
print("\n The best parameters across ALL searched params:\n", rf_random.best_params_)

# save the model
dump(rf_random.best_estimator_, f'best_model_{PREDICTOR_TYPE}.joblib')

### Random Search + Grid Search For All Predictors

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def evaluate(x_test, y_test, model):
    y_pred = model.predict(x_test)
    r2 = r2_score(y_test, y_pred)
    print("R2 Score: ", r2)
    return r2

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, r2_score

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 100)]

# Number of features to consider at every split
max_features = ['log2', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth = [int(x) for x in np.linspace(1, 20, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [54]:
import json
from joblib import dump

# for PREDICTOR_TYPE in ['MEM', 'EXF', 'LAN', 'VSP']:

PREDICTOR_TYPE = 'VSP'

# clean the data
y_train_cleaned = y_train[f'PHC_{PREDICTOR_TYPE}'].reset_index(drop=True)
y_test_cleaned =y_test[f'PHC_{PREDICTOR_TYPE}'].reset_index(drop=True)

nan_indices = y_train_cleaned.index[y_train_cleaned.isna()]
y_train_cleaned = y_train_cleaned.drop(nan_indices)
x_train_cleaned = np.delete(x_train, nan_indices, axis = 0)

nan_indices_test = y_test_cleaned.index[y_test_cleaned.isna()]
y_test_cleaned = y_test_cleaned.drop(nan_indices_test)
x_test_cleaned = np.delete(x_test, nan_indices_test, axis = 0)
# ==================================================================

# Use the random grid to search for best hyperparameters

# First create the base model to tune
rf = RandomForestRegressor()

# Evaluation metric
r2_scorer = make_scorer(r2_score)

# Random search of parameters, using 5 fold cross validation, 

# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(
    estimator = rf, param_distributions = random_grid, scoring=r2_scorer,
    n_iter = 100, cv = 5, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_random.fit(x_train_cleaned, y_train_cleaned)

# ======================================================================

best_model = rf_random.best_estimator_

print("\n The best estimator across ALL searched params:\n", best_model)
print("\n The best score across ALL searched params:\n", rf_random.best_score_)
print("\n The best parameters across ALL searched params:\n", rf_random.best_params_)

# save the model
dump(best_model, f'best_model_{PREDICTOR_TYPE}_random.joblib')

evaluate(x_test_cleaned, y_test_cleaned, best_model)

# Get the best parameter set
details = {}
PARAM_FILE = f'{PREDICTOR_TYPE}_best_params_random.json'

details['params'] = rf_random.best_params_
details['score'] = rf_random.best_score_

# Write data to a JSON file
with open(PARAM_FILE, 'w') as json_file:
    json.dump(details, json_file)
        

Fitting 5 folds for each of 100 candidates, totalling 500 fits

 The best estimator across ALL searched params:
 RandomForestRegressor(max_features='log2', min_samples_leaf=8,
                      min_samples_split=5, n_estimators=1635)

 The best score across ALL searched params:
 0.024976937716424886

 The best parameters across ALL searched params:
 {'n_estimators': 1635, 'min_samples_split': 5, 'min_samples_leaf': 8, 'max_features': 'log2', 'max_depth': None, 'bootstrap': True}
R2 Score:  0.002978908388633905
[CV] END bootstrap=False, max_depth=110, max_features=sqrt, min_samples_leaf=2, min_samples_split=2, n_estimators=925; total time= 4.4min
[CV] END bootstrap=True, max_depth=100, max_features=log2, min_samples_leaf=2, min_samples_split=2, n_estimators=1289; total time=  47.3s
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=1731; total time= 3.4min
[CV] END bootstrap=True, max_depth=80, max_features=sqrt, min_sampl

Grid Search to Improve Hyperparameters

In [ ]:
# Grid Search - EXF

#  {'n_estimators': 503, 'min_samples_split': 5, 'min_samples_leaf': 8, 'max_features': 'log2', 'max_depth': 10, 'bootstrap': True}

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [None, 5, 10],
    'max_features': ['log2'],
    'min_samples_leaf': [7, 8, 9, 10],
    'min_samples_split': [2, 3, 5, 7],
    'n_estimators': [503, 403, 603, 1503]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

grid_search.fit(x_train_cleaned, y_train_cleaned)

best_grid = grid_search.best_estimator_

print("\n The best estimator across ALL searched params:\n", best_grid)
print("\n The best score across ALL searched params:\n", grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n", grid_search.best_params_)

evaluate(x_test_cleaned, y_test_cleaned, best_grid)

# save the model
dump(best_grid, f'best_model_{PREDICTOR_TYPE}_grid_2.joblib')

# Get the best parameter set
details = {}
PARAM_FILE = f'{PREDICTOR_TYPE}_best_params_grid.json'

details['params'] = grid_search.best_params_
details['score'] = grid_search.best_score_

# Write data to a JSON file
with open(PARAM_FILE, 'w') as json_file:
    json.dump(details, json_file)

In [29]:
# best_grid = grid_search.best_estimator_

# print("\n The best estimator across ALL searched params:\n", best_grid)
# print("\n The best score across ALL searched params:\n", grid_search.best_score_)
# print("\n The best parameters across ALL searched params:\n", grid_search.best_params_)

# evaluate(x_test_cleaned, y_test_cleaned, best_grid)


 The best estimator across ALL searched params:
 RandomForestRegressor(max_features='log2', min_samples_leaf=10,
                      min_samples_split=5, n_estimators=2635)

 The best score across ALL searched params:
 0.052288095798545944

 The best parameters across ALL searched params:
 {'bootstrap': True, 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 10, 'min_samples_split': 5, 'n_estimators': 2635}
R2 Score:  0.07320436157894383


0.07320436157894383

[CV] END bootstrap=False, max_depth=40, max_features=sqrt, min_samples_leaf=8, min_samples_split=2, n_estimators=1213; total time= 3.4min
[CV] END bootstrap=True, max_depth=80, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=1059; total time= 3.5min
[CV] END bootstrap=False, max_depth=50, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=1788; total time=  36.1s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=8, min_samples_split=5, n_estimators=1884; total time= 3.2min
[CV] END bootstrap=True, max_depth=10, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=2000; total time= 3.5min
[CV] END bootstrap=True, max_depth=70, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=138; total time=   3.7s
[CV] END bootstrap=True, max_depth=70, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=138; total time=   3.9s
[CV] END bootstrap=True, max_depth=100,